# GLD Example

This notebook demonstrates how to generate groundwaterlevel research (GLD) related requests for data exchange with the bronhouderportaal (BRO). Also, it is shown how to validate these requests and deliver them to the bronhouderportaal. 

In [1]:
import bro_exchange as brx
from getpass import getpass

Define the token used for authentification (Bronhouderportaal API) below. Note: a token generated in the demo environment should be used. Besides, the project id should be given when using api v2.

In [2]:
user = input('user:')
password = getpass('pass:')
project_id = input('project_id:')

## 1. Register a new GLD registration

### 1.1. Generate registration request

Get GLD data:

In [3]:
monitoringpoints = [{'broId':'GMW000000042661',
                     
                   'tubeNumber':1}]

groundwatermonitoringnets = [{'broId':'GMN000000000083'},
                             {'broId':'GMN000000000084'}]


srcdocdata =  {'objectIdAccountableParty':'Meetpunt1',
            'groundwaterMonitoringNets':groundwatermonitoringnets, #     
            'monitoringPoints':monitoringpoints,        #
            }    


Generate registration request (GLD_StartRegistration):

In [20]:
# NOTE: DEFINE OUTPUT DIRECTORY:
reg = brx.gld_registration_request(srcdoc='GLD_StartRegistration', requestReference = '10_GLD_StartRegistration', deliveryAccountableParty = '27376655', qualityRegime = 'IMBRO', srcdocdata=srcdocdata)
reg.generate()
reg.write_request(output_dir = 'output\gld', filename = 'registration_request_gld_startregistration_test.xml')


### 1.2. Validate registration request

Validate the generated request (by means of bronhouderportaal api). In this case, the demo environment of the bronhouderportaal is used.

In [21]:
reg.validate(user=user, password = password, demo=True, api='v2', project_id = project_id)

Check if there are some errors:

In [22]:
print(reg.validation_status)
print(reg.validation_info)

VALIDE
{'status': 'VALIDE'}


The request seems to be valid

### 1.3. Deliver registration request

Deliver the generared request. 

In [23]:
reg.deliver(user=user, password=password, api='v2', project_id = 1255, demo=True)
print(reg.delivery_info.json())
print(reg.delivery_id)

{'identifier': '0000182628', 'status': 'AANGELEVERD', 'lastChanged': '2022-11-16T01:25:57.157241', 'brondocuments': [{'id': 418616, 'status': 'GELDIG', 'lastChanged': '2022-11-16T01:25:57.179702', 'filename': '79a19c2c6aa4.xml', 'requestReference': '10_GLD_StartRegistration'}]}
0000182628


After the request has been deliverd, the delivery id can be stored to track the status.

In [8]:
delivery_id = reg.delivery_id

### 1.4. Track status of pending delivery

To track the status of a pending delivery, the 'check_status' function of bro-exchange can be used. It only requires the delivery id, api version, environment and the authentication data. If api v2 is used, the project id should also be supplied:

In [24]:
status = brx.check_delivery_status(delivery_id, user=user, password=password, api='v2', project_id = project_id, demo=True)
print(status.json())

{'identifier': '0000182622', 'status': 'DOORGELEVERD', 'lastChanged': '2022-11-16T01:21:45.029849', 'brondocuments': [{'id': 418610, 'status': 'OPGENOMEN_LVBRO', 'lastChanged': '2022-11-16T01:21:44.790847', 'broId': 'GLD000000054499', 'transactionId': 'GLD-000000190176', 'filename': 'f56689867a92.xml', 'requestReference': '10_GLD_StartRegistration'}]}


If the delivery has been accepted, the BRO id can be retrieved from the status:

In [25]:
broid_gld = status.json()['brondocuments'][0]['broId']
print(broid_gld)

GLD000000054499


## 2. Add groundwaterlevel data to GLD

### 2.1. Generating addition request

Get groundwaterlevel data. Three type of timeseries can be distinguished: regulier voorlopig, regulier volledig beoordeeld, controlemeting.

In [32]:
# Parameters voorlopig & volledig beoordeeld:
metadata_parameters_1= {'principalInvestigator':'27376655', # kvk principalinvestigator
            'observationType':'reguliereMeting'
             }

procedure_parameters_1 = {
                        'evaluationProcedure':'oordeelDeskundige',
                        'measurementInstrumentType':'akoestischeSensor'   
                        }

# Parameters controlemeting:
metadata_parameters_2= {'principalInvestigator':'27376655',
            'observationType':'controlemeting'
             }



procedure_parameters_2 = {'evaluationProcedure':'oordeelDeskundige',
                        'measurementInstrumentType':'analoogPeilklokje'   
                        }

# Timeseriesdata (Eenheid is verplicht meter tov NAP):
results_voorlopig = [{'time':'2019-01-07T08:14:38+01:00','value':-4.345,'metadata':{'StatusQualityControl':'goedgekeurd','interpolationType':'Discontinuous'}},#'censoredReason':}}
                    {'time':'2019-01-21T10:01:52+01:00','value':'None','metadata':{'StatusQualityControl':'afgekeurd','interpolationType':'Discontinuous','censoringLimitvalue':0,'censoredReason':'BelowDetectionRange'}} ,                      
                    {'time':'2019-01-28T16:58:07+01:00','value':-4.788,'metadata':{'StatusQualityControl':'goedgekeurd','interpolationType':'Discontinuous'}},#'censoredReason':}}                        
                    ]

results_controle = [{'time':'2018-04-15T08:05:42+02:00','value':-4.350,'metadata':{'StatusQualityControl':'goedgekeurd','interpolationType':'Discontinuous'}}
                    ]

results_volledig_beoordeeld = [{'time':'2019-01-07T08:14:38+01:00','value':-4.345,'metadata':{'StatusQualityControl':'goedgekeurd','interpolationType':'Discontinuous'}},#'censoredReason':}}
                    {'time':'2019-01-21T10:01:52+01:00','value':-4.987,'metadata':{'StatusQualityControl':'goedgekeurd','interpolationType':'Discontinuous'}},#'censoredReason':}}                        
                    {'time':'2019-01-28T16:58:07+01:00','value':-4.788,'metadata':{'StatusQualityControl':'goedgekeurd','interpolationType':'Discontinuous'}},#'censoredReason':}}                        
                    ]

    
# Input arguments sourcedocs additions (voorlopig, volledigBeoordeeld & controlemeting)  
arglist_voorlopig =  {'metadata':{'status':'voorlopig','parameters':metadata_parameters_1,'dateStamp':'2019-01-28'},
                        'procedure':{'parameters':procedure_parameters_1}, 
                        'resultTime':'2019-01-28T16:58:07+01:00',
                        'result':results_voorlopig, 
                        }    

arglist_controle =  {'metadata':{'parameters':metadata_parameters_2,'dateStamp':'2018-04-15'}, 
                        'procedure':{'parameters':procedure_parameters_2},
                        'resultTime':'2018-04-15T08:05:42+02:00',
                        'result':results_controle, 
                        }  

arglist_volledig_beoordeeld =  {'metadata':{'status':'volledigBeoordeeld','parameters':metadata_parameters_1,'dateStamp':'2019-05-28'},
                                    'procedure':{'parameters':procedure_parameters_1},
                                    'resultTime':'2019-05-28T16:58:07+01:00', # Resultaat volledig beoordeeld is als de validatie is afgerond
                                    'result':results_volledig_beoordeeld, 
                                    }   


Create and validate the registration requests (gld_additions):

In [33]:
# voorlopig
print("Registration request partly validated series:\n")
reg_voorlopig = brx.gld_registration_request(srcdoc='GLD_Addition', requestReference = '11_GLD_Addition_sensorisch', deliveryAccountableParty = '27376655', qualityRegime = 'IMBRO',broId=broid_gld, srcdocdata=arglist_voorlopig)
reg_voorlopig.generate()
reg_voorlopig.write_request(output_dir = 'output\gld', filename = 'registration_request_gld_addition_voorlopig_test.xml')

reg_voorlopig.validate(user=user, password = password, demo=True, api='v2', project_id = project_id)

print(reg_voorlopig.validation_status)
print(reg_voorlopig.validation_info)

print('-------------------\n')

# controle
print("Registration request verification series:\n")
reg_controle = brx.gld_registration_request(srcdoc='GLD_Addition', requestReference = '11_GLD_Addition_sensorisch', deliveryAccountableParty = '27376655', qualityRegime = 'IMBRO',broId=broid_gld, srcdocdata=arglist_controle)
reg_controle.generate()
reg_controle.write_request(output_dir = 'output\gld', filename = 'registration_request_gld_addition_controle_test.xml')

reg_controle.validate(user=user, password = password, demo=True, api='v2', project_id = project_id)
print(reg_controle.validation_status)
print(reg_controle.validation_info)

print('-------------------\n')
# volledig
print("Registration request validated series:\n")
reg_volledig = brx.gld_registration_request(srcdoc='GLD_Addition', requestReference = '11_GLD_Addition_sensorisch', deliveryAccountableParty = '27376655', qualityRegime = 'IMBRO',broId=broid_gld, srcdocdata=arglist_volledig_beoordeeld)
reg_volledig.generate()
reg_volledig.write_request(output_dir = 'output\gld', filename = 'registration_request_gld_addition_volledig_test.xml')

payload = reg_voorlopig.request

reg_volledig.validate(user=user, password = password, demo=True, api='v2', project_id = project_id)
print(reg_volledig.validation_status)
print(reg_volledig.validation_info)
print('-------------------\n')


Registration request partly validated series:

VALIDE
{'status': 'VALIDE'}
-------------------

Registration request verification series:

VALIDE
{'status': 'VALIDE'}
-------------------

Registration request validated series:

VALIDE
{'status': 'VALIDE'}
-------------------



Deliver:

In [34]:
reg_voorlopig.deliver(user=user, password=password, api='v2', project_id = 1255, demo=True)
print(reg_voorlopig.delivery_info.json())
print(reg_voorlopig.delivery_id)

print('-------------------\n')

reg_controle.deliver(user=user, password=password, api='v2', project_id = 1255, demo=True)
print(reg_controle.delivery_info.json())
print(reg_controle.delivery_id)

print('-------------------\n')

reg_volledig.deliver(user=user, password=password, api='v2', project_id = 1255, demo=True)
print(reg_volledig.delivery_info.json())
print(reg_volledig.delivery_id)
print('-------------------\n')

{'identifier': '0000182629', 'status': 'AANGELEVERD', 'lastChanged': '2022-11-16T01:28:38.51379', 'brondocuments': [{'id': 418617, 'status': 'GELDIG', 'lastChanged': '2022-11-16T01:28:38.537643', 'filename': 'ee9d761627d7.xml', 'requestReference': '11_GLD_Addition_sensorisch'}]}
0000182629
-------------------

{'identifier': '0000182630', 'status': 'AANGELEVERD', 'lastChanged': '2022-11-16T01:28:39.446717', 'brondocuments': [{'id': 418618, 'status': 'GELDIG', 'lastChanged': '2022-11-16T01:28:39.470468', 'filename': '21f0677b6651.xml', 'requestReference': '11_GLD_Addition_sensorisch'}]}
0000182630
-------------------

{'identifier': '0000182631', 'status': 'AANGELEVERD', 'lastChanged': '2022-11-16T01:28:40.414038', 'brondocuments': [{'id': 418619, 'status': 'GELDIG', 'lastChanged': '2022-11-16T01:28:40.436085', 'filename': '9e7fc463bc6a.xml', 'requestReference': '11_GLD_Addition_sensorisch'}]}
0000182631
-------------------



Check status:

In [36]:
status = brx.check_delivery_status(reg_voorlopig.delivery_id, user=user, password=password, api='v2', project_id = project_id, demo=True)
print(status.json())

status = brx.check_delivery_status(reg_controle.delivery_id, user=user, password=password, api='v2', project_id = project_id, demo=True)
print(status.json())

status = brx.check_delivery_status(reg_volledig.delivery_id, user=user, password=password, api='v2', project_id = project_id, demo=True)
print(status.json())

{'identifier': '0000182629', 'status': 'DOORGELEVERD', 'lastChanged': '2022-11-16T01:28:43.369737', 'brondocuments': [{'id': 418617, 'status': 'OPGENOMEN_LVBRO', 'lastChanged': '2022-11-16T01:28:43.09891', 'broId': 'GLD000000054499', 'transactionId': 'GLD-000000190183', 'filename': 'ee9d761627d7.xml', 'requestReference': '11_GLD_Addition_sensorisch'}]}
{'identifier': '0000182630', 'status': 'DOORGELEVERD', 'lastChanged': '2022-11-16T01:28:47.254766', 'brondocuments': [{'id': 418618, 'status': 'OPGENOMEN_LVBRO', 'lastChanged': '2022-11-16T01:28:47.025394', 'broId': 'GLD000000054499', 'transactionId': 'GLD-000000190184', 'filename': '21f0677b6651.xml', 'requestReference': '11_GLD_Addition_sensorisch'}]}
{'identifier': '0000182631', 'status': 'DOORGELEVERD', 'lastChanged': '2022-11-16T01:29:10.310778', 'brondocuments': [{'id': 418619, 'status': 'OPGENOMEN_LVBRO', 'lastChanged': '2022-11-16T01:29:10.090622', 'broId': 'GLD000000054499', 'transactionId': 'GLD-000000190185', 'filename': '9e7f